In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
c = pd.read_csv(f"../assets/annotated-corpus/train/1.tsv",delimiter='\t',keep_default_na=False)
lemmas = []
sentence_lemmas=[]
for el in  tqdm(c.values):
    if el[1]!="<endofsentence>":
        sentence_lemmas.append(el[3])
    else:
        lemmas.append(sentence_lemmas)
        sentence_lemmas=[]


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words('english'))


In [ ]:
import re
filtered_lemmas = []
for sentence in lemmas:
    filtered_lemmas.append([])
    for lemma in sentence:
        if type(lemma) == str:
            lemma_filtered = re.sub(r'[^\w\s]','', lemma)
            if len(lemma_filtered)==0 or lemma_filtered in stops:
                continue
            filtered_lemmas[-1].append(lemma_filtered)

In [ ]:
# from nltk.util import ngrams
import itertools as itertools
from collections import Counter 

word_cnt = Counter()
trigram_cnt = Counter()
for sentence in filtered_lemmas:
    for i in range(len(sentence)-2):
        w1,w2,w3 = sentence[i], sentence[i+1], sentence[i+2]
        word_cnt[w1] += 1
        trigram_cnt[(w1, w2, w3)] += 1

In [ ]:
N_total = len(word_cnt)

In [ ]:
def score_ngram(score_fn, w1, w2, w3):
    ngram_cnt = trigram_cnt[(w1, w2, w3)]
    if not ngram_cnt:
        return
    unigrams = (word_cnt[w1],word_cnt[w2],word_cnt[w3])
    return score_fn(ngram_cnt,unigrams)

def _score_ngrams(score_fn):
    for tup in trigram_cnt:
        score = score_ngram(score_fn, *tup)
        if score is not None:
            yield tup, score

def score_ngrams(score_fn):
    return sorted(_score_ngrams(score_fn), key=lambda t: (-t[1], t[0]))

def nbest(score_fn,n):
    return [p for p, s in score_ngrams(score_fn)[:n]]

In [ ]:
def student_t(ngram_cnt,unigrams):
    unigrams_product = 1
    for el in unigrams:
        unigrams_product*=el
    return (ngram_cnt - (unigrams_product / (N_total ** (3 - 1)))) / (ngram_cnt + 1e-5) ** 0.5

In [ ]:
nbest(student_t,50)

In [ ]:
import nltk
from nltk.collocations import *

trigram_measures = nltk.collocations.TrigramAssocMeasures()

flat_list = [item for sublist in filtered_lemmas for item in sublist]
finder_thr = TrigramCollocationFinder.from_words(flat_list) 

In [ ]:
print(finder_thr.nbest(trigram_measures.student_t, 50))